# 1. 개발 환경 설정

### 1.1 필수 라이브러리 설치하기

In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 21.4 MB/s eta 0:00:00
   ━━━━━

### 1.2 Import modules

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

### 1.3 Huggingface 로그인

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# 2. Dataset 생성 및 준비

### 2.1 데이터셋 로드

In [ ]:
from datasets import load_dataset
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### 2.2 데이터셋 탐색

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

### 2.3 데이터셋 예시

In [ ]:
dataset['train'][0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

# 3. Gemma 모델의 한국어 요약 테스트

### 3.1 모델 로드

In [ ]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-66e93cf5-37cea5bb1b83b8356326e91a;867c56a3-d80d-4fe4-a773-7c1a168bb2e7)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in.

### 3.2 Gemma-it의 프롬프트 형식

In [ ]:
doc = dataset['train']['document'][0]

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요 :\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
prompt

### 3.3 Gemma-it 추론

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

# 4. Gemma 파인튜닝

#### 주의: Colab GPU 메모리 한계로 이전장 추론에서 사용했던 메모리를 비워 줘야 파인튜닝을 진행 할 수 있습니다. <br> notebook 런타임 세션을 재시작 한 후 1번과 2번의 2.1 항목까지 다시 실행하여 로드 한 후 아래 과정을 진행합니다

In [ ]:
# !nvidia-smi

### 4.1 학습용 프롬프트 조정

In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['document'][i], example['summary'][i]))
    return prompt_list

In [ ]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
다음 글을 요약해주세요:

앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.<end_of_turn>
<start_of_turn>model
올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정한 가운데

### 4.2 QLoRA 설정

In [ ]:
lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
BASE_MODEL = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### 4.3 Trainer 실행

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        # max_steps=3000,
        max_steps=50,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/22194 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=50, training_loss=2.6769122314453124, metrics={'train_runtime': 90.681, 'train_samples_per_second': 2.206, 'train_steps_per_second': 0.551, 'total_flos': 1144942188429312.0, 'train_loss': 2.6769122314453124, 'epoch': 0.01})

### 4.4 Finetuned Model 저장

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
!ls -alh lora_adapter

total 29M
drwxr-xr-x 2 root root 4.0K Sep 19 08:44 .
drwxr-xr-x 1 root root 4.0K Sep 19 08:44 ..
-rw-r--r-- 1 root root  689 Sep 19 08:44 adapter_config.json
-rw-r--r-- 1 root root  29M Sep 19 08:44 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Sep 19 08:44 README.md


In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-sum-ko')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!ls -alh ./gemma-2b-it-sum-ko

total 4.7G
drwxr-xr-x 2 root root 4.0K Sep 19 08:44 .
drwxr-xr-x 1 root root 4.0K Sep 19 08:44 ..
-rw-r--r-- 1 root root  662 Sep 19 08:44 config.json
-rw-r--r-- 1 root root  132 Sep 19 08:44 generation_config.json
-rw-r--r-- 1 root root 4.7G Sep 19 08:44 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  65M Sep 19 08:44 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  14K Sep 19 08:44 model.safetensors.index.json


# 5. Gemma 한국어 요약 모델 추론

#### 주의: 마찬가지로 Colab GPU 메모리 한계로 학습 시 사용했던 메모리를 비워 줘야 파인튜닝을 진행 할 수 있습니다. <br> notebook 런타임 세션을 재시작 한 후 1번과 2번의 2.1 항목까지 다시 실행하여 로드 한 후 아래 과정을 진행합니다

In [ ]:
!nvidia-smi

Thu Sep 19 08:44:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   76C    P0              35W /  72W |   8549MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 5.1 Fine-tuned 모델 로드

In [ ]:
BASE_MODEL = "google/gemma-2b-it"
FINETUNE_MODEL = "./gemma-2b-it-sum-ko"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### 5.2 Fine-tuned 모델 추론

In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
doc = dataset['test']['document'][10]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
print(doc)

사진 한국동서발전 한국동서발전은 다음달 31일까지 울산시민과 함께하는 생활 속 걷기 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행한다고 1일 밝혔다. 울산숲사랑운동과 함께 하는 이번 챌린지는 코로나19로 지친 시민들이 일상 속 걷기 운동을 통해 건강을 증진하고 자가용 대신 대중교통을 이용해 탄소중립 사회를 실현하기 위해 마련됐다. 울산시민 누구나 구글플레이 또는 앱스토어에서 스마트폰 어플 워크온 을 내려받아 회원가입 후 참여할 수 있다. 챌린지 목표인 10만보·20만보·30만보를 기간 내 달성하면 추첨을 통해 교통카드 등 소정의 기념품을 증정한다. 김태규 동서발전 탄소중립실장은 초록발자국 챌린지를 통해 코로나19로 지친 울산시민들이 활력을 되찾고 대중교통 활성화로 탄소중립 실천에 기여하길 바란다 며 앞으로도 다양한 탄소중립·환경 프로젝트를 추진하여 더 많은 시민이 참여할 수 있도록 최선을 다하겠다 고 말했다.


In [ ]:
print(messages)

[{'role': 'user', 'content': '다음 글을 요약해주세요:\n\n사진 한국동서발전 한국동서발전은 다음달 31일까지 울산시민과 함께하는 생활 속 걷기 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행한다고 1일 밝혔다. 울산숲사랑운동과 함께 하는 이번 챌린지는 코로나19로 지친 시민들이 일상 속 걷기 운동을 통해 건강을 증진하고 자가용 대신 대중교통을 이용해 탄소중립 사회를 실현하기 위해 마련됐다. 울산시민 누구나 구글플레이 또는 앱스토어에서 스마트폰 어플 워크온 을 내려받아 회원가입 후 참여할 수 있다. 챌린지 목표인 10만보·20만보·30만보를 기간 내 달성하면 추첨을 통해 교통카드 등 소정의 기념품을 증정한다. 김태규 동서발전 탄소중립실장은 초록발자국 챌린지를 통해 코로나19로 지친 울산시민들이 활력을 되찾고 대중교통 활성화로 탄소중립 실천에 기여하길 바란다 며 앞으로도 다양한 탄소중립·환경 프로젝트를 추진하여 더 많은 시민이 참여할 수 있도록 최선을 다하겠다 고 말했다.'}]


In [ ]:
print(prompt)

<bos><start_of_turn>user
다음 글을 요약해주세요:

사진 한국동서발전 한국동서발전은 다음달 31일까지 울산시민과 함께하는 생활 속 걷기 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행한다고 1일 밝혔다. 울산숲사랑운동과 함께 하는 이번 챌린지는 코로나19로 지친 시민들이 일상 속 걷기 운동을 통해 건강을 증진하고 자가용 대신 대중교통을 이용해 탄소중립 사회를 실현하기 위해 마련됐다. 울산시민 누구나 구글플레이 또는 앱스토어에서 스마트폰 어플 워크온 을 내려받아 회원가입 후 참여할 수 있다. 챌린지 목표인 10만보·20만보·30만보를 기간 내 달성하면 추첨을 통해 교통카드 등 소정의 기념품을 증정한다. 김태규 동서발전 탄소중립실장은 초록발자국 챌린지를 통해 코로나19로 지친 울산시민들이 활력을 되찾고 대중교통 활성화로 탄소중립 실천에 기여하길 바란다 며 앞으로도 다양한 탄소중립·환경 프로젝트를 추진하여 더 많은 시민이 참여할 수 있도록 최선을 다하겠다 고 말했다.<end_of_turn>
<start_of_turn>model



In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

한국동서발전은 다음달 31일까지 울산시민과 함께 하는 생활 속 걷기 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행한다고 1일 밝혔으며 울산숲사랑운동과 함께 하는 이번 챌린지는 코로나19로 지친 시민들이 일상 속 걷기 운동을 통해 건강을 증진하고 자가용 대신 대중교통을 이용해 탄소중립 사회를 실현하기 위해 마련됐다.
